In [2]:
import sys
import os  
from datetime import datetime, timedelta, date
import datetime as dt

import requests
import pandas as pd
import json
import yaml
import importlib


from snowflake.snowpark.functions import date_trunc, current_date, to_date, month
from snowflake.snowpark.functions import col, lit, when
from snowflake.snowpark.types import DateType
import snowflake.snowpark.functions as f


sys.path.append(os.path.abspath('scripts'))

try:
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
except Exception:
    import scripts.SnowflakeConnector  # brings module into scope
    importlib.reload(scripts.SnowflakeConnector)
    from scripts.SnowflakeConnector import create_active_session
    session = create_active_session()


import scripts.utils
importlib.reload(scripts.utils)
from scripts.utils import get_orders_for_date, get_max_date_from_table


In [3]:
#Access Token
TOKEN_PATH = "config/shopify_auth.yaml"
QUERY_PATH = "graphql/queries/get_sellingplan.graphql"
SELLINGPLAN_TABLE = "snowpark_db.shopify.orders_with_selling_plan"
DATE_COLUMN = "PROCESSED_AT"


with open(TOKEN_PATH, "r") as file:
    config = yaml.safe_load(file)
ACCESS_TOKEN = config.get("ACCESS_TOKEN", {})


with open(QUERY_PATH, "r") as file:
    QUERY_STRING = file.read()


SHOP_ENDPOINT = "https://shopmbg.myshopify.com/admin/api/2024-04/graphql.json"


In [5]:
#maxdate in SELLINGPLAN_TABLE
today = date.today()
print(today)

2025-06-30


In [7]:
load_date = today - timedelta(days=1)
load_date = load_date.strftime("%Y-%m-%d")


all_orders = get_orders_for_date(load_date, ACCESS_TOKEN, SHOP_ENDPOINT, QUERY_STRING)
now = datetime.utcnow()
paginated_df = pd.DataFrame([
        {
            "PROCESSED_AT" :o['node']['processedAt'] ,
            "ORDER_ID": o['node']['id'].split('/')[-1],
            "ORDER_NAME": o['node']['name'].split('#')[-1],
            "LINEITEM_ID":json.dumps(o['node']['lineItems']),
            "TAGS":json.dumps(o['node']['tags']),
            "LOAD_TIMESTAMP": now
        }
        for o in all_orders
    ])

end_date = datetime.strptime(load_date, "%Y-%m-%d") + timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%dT00:00:00Z')
paginated_df = paginated_df[paginated_df["PROCESSED_AT"]<end_date]


# paginated_snow_df = session.create_dataframe(paginated_df)
# paginated_snow_df.write.save_as_table(SELLINGPLAN_TABLE, mode="append")


Fetched 1188 orders for 2025-06-29
